In [1]:
import re
import datetime
import gc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [2]:
PATH_DATA = "../../../data/"

# 1. Парсим логи Radius'а

In [6]:
def read_data(filename):
    # 'radius.log.1'
    with open(PATH_DATA + filename, 'rb') as f:
        rows = f.readlines()
    rows = [str(row)[2:][:-3] for row in tqdm(rows)]
    return rows

In [7]:
def unlist(x: list):
    if len(x) > 0:
        return x[0]
    else:
        return ''

In [8]:
def parse_file(filename): 
    columns = [
        'date',
#         'rad/nfs',
#         'radiusd',
        'type_',
#         'id_сессии',
#         'flag_мастер_сессия',
        'сессия_аккаунтинга_направления',
        'логин_абонента',
        'ip_абонента',
        'mac_абонента',
        'start_stop_alige',
        'status_ok_or_not',
        'БРАС-1/2',
        'delay',
        'input_trafic', 
        'output_trafic',
#         'row'
    ]
    res_parse = []
    bad_parse_rows = []
    len_date = len("Apr 30 00:11:25")
    
    with open(PATH_DATA + filename, 'rb') as f:
        for idx, row_ in tqdm(enumerate(f)):
            row = str(row_)[2:][:-3]
            if 'huawei_save_sessid' in row:
                continue
#             if idx > 1_000_000:
#                 break

            date = row[:len_date]
#             rad_nfs = row[(len_date+1): (len_date+4)]
#             radiusd = unlist(re.findall(r'radiusd: \[(\w+)\]', row))
            type_ = unlist(re.findall(r'radiusd: \[\w+\] ([\w+|-|_|-]+)', row))
            if type_ not in ['Billing-Authentication', 'Billing-Accounting']:
                continue
                
        
#             ulsk = re.findall(r'ULSK-(\w+)', row)
#             flg_master_session = "RADIUS,," in row
            session_accountinga_napravlenia = unlist(re.findall(r'RADIUS,(\d),', row))
#             login_abonenta = unlist(re.findall(r"'(.+)' via", row))
            login_abonenta = unlist(re.findall(r" ([\S]+@[\S]+) ", row)).replace("'", "").replace("\\", "")
        
            ip_abonent = unlist(re.findall(r", (\d+.\d+.\d+.\d+), ", row))
            mac_abonent = unlist(re.findall(r", (\w+:\w+:\w+:\w+:\w+:\w+)\) ", row))
            start_stop_alige = unlist(re.findall(r",PPP,(\w+),", row))
            status_ok_or_not = unlist(re.findall(r"- (.+) --", row))[:60]
            bras_1_2 = unlist(re.findall(r"89.239.189.1:\d+|89.239.189.2:\d+", row))
#                     '2020.04.30 00:11:25,89.239.189.1:153223559' 89.239.189.2:153224753
            delay = unlist(re.findall(r"delay (\d+\.\d+)", row))
            input_trafic = unlist(re.findall(r',\d+.\d+.\d+.\d+,\d+,\d+,(\d+),\d+, ', row))
            output_trafic = unlist(re.findall(r',\d+.\d+.\d+.\d+,\d+,\d+,\d+,(\d+), ', row))

#             if (delay != '') and (input_trafic == ''):
# #                 pass
#                 bad_parse_rows.append(row)
#             else:
            res_parse.append([
                date, type_, session_accountinga_napravlenia,
                login_abonenta, ip_abonent, mac_abonent, start_stop_alige,
                status_ok_or_not, bras_1_2, delay, input_trafic, output_trafic,
#                 row
            ])
        
    df_ = pd.DataFrame(res_parse, columns=columns)
    df_['date'] = pd.to_datetime('2020' + df_['date'], format='%Y%b %d %H:%M:%S')
    
    return bad_parse_rows, df_

In [9]:
bad_parse_rows, a = parse_file(f'radius.log.{1}')

In [10]:
a['status_ok_or_not'].value_counts()

OK                                                              6214367
OK Skiped due to traffic limit                                   441653
FAILED Service isn?t ON.                                          49599
FAILED Authentication failure:"Password match failure"             1716
FAILED Tarification error.<SQL>ORA-20001: ORA-20099: PKG-000       1493
FAILED Not enough money on account.                                 832
FAILED Authentication failure:"no Password or CHAP value-pai          8
FAILED No username found                                              8
Name: status_ok_or_not, dtype: int64

In [12]:
a['status_ok_or_not'].value_counts()

OK                                                              6602092
OK Skiped due to traffic limit                                   503037
FAILED Service isn?t ON.                                          47873
FAILED Not enough money on account.                                3421
FAILED Authentication failure:"Password match failure"             1421
FAILED Tarification error.<SQL>ORA-20001: ORA-20099: PKG-000       1177
FAILED Authentication failure:"no Password or CHAP value-pai         80
FAILED STOP on NORMAL closed session.                                10
FAILED Session in work.                                               7
FAILED No username found                                              6
FAILED ALIVE on NORMAL closed session.                                1
Name: status_ok_or_not, dtype: int64

In [15]:
a.head()

,date,type_,сессия_аккаунтинга_направления,логин_абонента,ip_абонента,mac_абонента,start_stop_alige,status_ok_or_not,БРАС-1/2,delay,input_trafic,output_trafic
0,2020-05-02 00:11:14,Billing-Accounting,,rSTKIoB9@pppoe,89.239.189.2,78:c1:a7:26:2b:f2,Alive,OK,89.239.189.2:35775804,0.357546,10804589355,171926012896
1,2020-05-02 00:11:14,Billing-Accounting,1,GBRFnJAM@pppoe,89.239.189.2,ec:08:6b:b7:72:1a,Alive,OK,89.239.189.2:253891725,0.273068,2989776,73828767
2,2020-05-02 00:11:14,Billing-Accounting,,OAXHLWA9@pppoe,89.239.189.1,d4:bf:7f:85:c8:01,Alive,OK,89.239.189.1:102888519,0.240214,5977761900,50271361952
3,2020-05-02 00:11:14,Billing-Accounting,6,GBRFnJAM@pppoe,89.239.189.2,ec:08:6b:b7:72:1a,Alive,OK,89.239.189.2:253891725,0.280033,9250671,303541285
4,2020-05-02 00:11:14,Billing-Accounting,6,Wssgg9-W@pppoe,89.239.189.2,ec:4c:4d:c1:51:d2,Alive,OK,89.239.189.2:102888473,0.282596,2394997,79303236


In [12]:
a.to_csv(f'parsed_radius_log_1.csv')

In [16]:
for i in range(2, 10):
    print(i)
    del a
    gc.collect()
    _, a = parse_file(f'radius.log.{i}')
    gc.collect()
    a.to_csv(f'parsed_radius_log_{i}.csv')

2



3



4



5



6



7



8



9


In [103]:
bad_parse_rows, a = parse_file('radius.log.1')

# 2. Агрегируем по часам

In [4]:
dfs = []
for i in tqdm(range(1, 10)):
    dfs.append(
        pd.read_csv(f'parsed_radius_log_{i}.csv')
        [['date', 'логин_абонента', 'type_', 'start_stop_alige', 'status_ok_or_not', 'delay', 'input_trafic', 'output_trafic']]
    )
dfs_concat = pd.concat(dfs)

In [14]:
date_ = pd.to_datetime(dfs_concat['date'])

dfs_concat['hour'] = date_.dt.hour
dfs_concat['day'] = date_.dt.day
dfs_concat['month'] = date_.dt.month

dfs_concat['delay'] = dfs_concat['delay'].astype(float)

In [45]:
dfs_concat['type_'].value_counts()

Billing-Accounting        61904357
Billing-Authentication     5322969
Name: type_, dtype: int64

In [49]:
dfs_concat.isnull().sum(axis=0)

date                       0
type_                      0
start_stop_alige    10511918
status_ok_or_not           0
delay                5188949
input_trafic        21033139
output_trafic       21033139
hour                       0
day                        0
month                      0
dtype: int64

In [48]:
dfs_concat.shape, dfs_concat.dropna().shape,

((67227326, 10), (46194187, 10))

In [50]:
gb_count = (
    dfs_concat
    .fillna('')
    .groupby(['month', 'day', 'hour', 'type_', 'start_stop_alige', 'status_ok_or_not'])
    [['date']]
    .count()
    .rename(columns={'date': 'count'})
    .reset_index()
)

### кол-во запросов

In [51]:
gb_count['date'] = [
    datetime.datetime(2020, month, day, hour)
    for month, day, hour in gb_count[['month', 'day', 'hour']].values
]

In [55]:
gb_count_pivot = pd.merge(
    (
        gb_count
        .pivot_table(index='date', columns='start_stop_alige', values='count', aggfunc='sum')
        .add_prefix('start_stop_alige_count_')
    ),
    (
        gb_count
        .pivot_table(index='date', columns='type_', values='count', aggfunc='sum')
        .add_prefix('type__count_')),
    left_index=True,
    right_index=True
)

In [57]:
gb_count_pivot.head(2)

,start_stop_alige_count_,start_stop_alige_count_Alive,start_stop_alige_count_Start,start_stop_alige_count_Stop,type__count_Billing-Accounting,type__count_Billing-Authentication
date,,,,,,
2020-04-22 00:00:00,33085,179362,22712,22388,239344,18203
2020-04-22 01:00:00,38363,209859,25201,19882,271724,21581


In [61]:
gb_count.head()

,month,day,hour,type_,start_stop_alige,status_ok_or_not,count,date
0,4,22,0,Billing-Accounting,,OK Skiped due to traffic limit,14882,2020-04-22
1,4,22,0,Billing-Accounting,Alive,OK,179362,2020-04-22
2,4,22,0,Billing-Accounting,Start,OK,22712,2020-04-22
3,4,22,0,Billing-Accounting,Stop,FAILED Tarification error.<SQL>ORA-20001: ORA-...,34,2020-04-22
4,4,22,0,Billing-Accounting,Stop,OK,22354,2020-04-22


### mean_delay

In [65]:
gb_mean_delay = (
    dfs_concat
    [['month', 'day', 'hour', 'delay']]
    .dropna()
    .groupby(['month', 'day', 'hour'])
    [['delay']]
    .mean()
    .rename(columns={'delay': 'mean_delay'})
    .reset_index()
)
gb_mean_delay['date'] = [
    datetime.datetime(2020, month, day, hour)
    for month, day, hour in gb_mean_delay[['month', 'day', 'hour']].values
]
gb_mean_delay = gb_mean_delay[['date', 'mean_delay']].set_index('date')

In [66]:
gb_mean_delay.head()

,mean_delay
date,
2020-04-22 00:00:00,0.269718
2020-04-22 01:00:00,0.218330
2020-04-22 02:00:00,0.216456
2020-04-22 03:00:00,0.212232
2020-04-22 04:00:00,0.214075


### mean_trafic

In [69]:
gb_mean_trafic = (
    dfs_concat
    [['month', 'day', 'hour', 'input_trafic', 'output_trafic']]
    .dropna()
    .groupby(['month', 'day', 'hour'])
    [['input_trafic', 'output_trafic']]
    .mean()
    .add_prefix('mean_')
    .reset_index()
)
gb_mean_trafic['date'] = [
    datetime.datetime(2020, month, day, hour)
    for month, day, hour in gb_mean_trafic[['month', 'day', 'hour']].values
]
gb_mean_trafic = gb_mean_trafic[['date', 'mean_input_trafic', 'mean_output_trafic']].set_index('date')

In [70]:
gb_mean_trafic.head()

,mean_input_trafic,mean_output_trafic
date,,
2020-04-22 00:00:00,2.907789e+09,1.262401e+10
2020-04-22 01:00:00,2.943544e+09,1.333590e+10
2020-04-22 02:00:00,3.674643e+09,1.501675e+10
2020-04-22 03:00:00,2.706126e+09,1.293037e+10
2020-04-22 04:00:00,4.032782e+09,1.616832e+10


### failed

In [83]:
df_failed = dfs_concat[['month', 'day', 'hour', 'status_ok_or_not']]
df_failed['failed'] = [int('FAILED' in x) for x in df_failed['status_ok_or_not']]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [87]:
gb_failed_count = (
    df_failed
    [['month', 'day', 'hour', 'failed']]
    .groupby(['month', 'day', 'hour'])
    [['failed']]
    .sum()
    .add_prefix('count_')
    .reset_index()
)
gb_failed_count['date'] = [
    datetime.datetime(2020, month, day, hour)
    for month, day, hour in gb_failed_count[['month', 'day', 'hour']].values
]
gb_failed_count = gb_failed_count[['date', 'count_failed']].set_index('date')

In [89]:
gb_failed_count.head()

,count_failed
date,
2020-04-22 00:00:00,2011
2020-04-22 01:00:00,2517
2020-04-22 02:00:00,2487
2020-04-22 03:00:00,2536
2020-04-22 04:00:00,2492


In [82]:
dfs_concat['status_ok_or_not'].value_counts()

OK                                                              61478815
OK Skiped due to traffic limit                                   5188949
FAILED Service isn?t ON.                                          495751
FAILED Not enough money on account.                                25340
FAILED Authentication failure:"Password match failure"             22086
FAILED Tarification error.<SQL>ORA-20001: ORA-20099: PKG-000       11874
FAILED STOP on NORMAL closed session.                               3500
FAILED Authentication failure:"no Password or CHAP value-pai         577
FAILED No username found                                             323
FAILED Doubling session in TB_WTMPS.                                  80
FAILED Session in work.                                               19
FAILED ALIVE on NORMAL closed session.                                11
FAILED Tarification error.<SQL>ORA-20001: ORA-00060: deadloc           1
Name: status_ok_or_not, dtype: int64

### Итого

In [90]:
gb_total = pd.concat(
    [gb_count_pivot, gb_mean_delay, gb_mean_trafic, gb_failed_count],
    axis=1
).sort_index()

In [91]:
gb_total.shape, gb_total.dropna().shape

((217, 10), (217, 10))

In [92]:
gb_total.head()

,start_stop_alige_count_,start_stop_alige_count_Alive,start_stop_alige_count_Start,start_stop_alige_count_Stop,type__count_Billing-Accounting,type__count_Billing-Authentication,mean_delay,mean_input_trafic,mean_output_trafic,count_failed
date,,,,,,,,,,
2020-04-22 00:00:00,33085,179362,22712,22388,239344,18203,0.269718,2.907789e+09,1.262401e+10,2011
2020-04-22 01:00:00,38363,209859,25201,19882,271724,21581,0.218330,2.943544e+09,1.333590e+10,2517
2020-04-22 02:00:00,37253,169129,24801,15142,225489,20836,0.216456,3.674643e+09,1.501675e+10,2487
2020-04-22 03:00:00,37810,155948,29919,16784,219988,20473,0.212232,2.706126e+09,1.293037e+10,2536
2020-04-22 04:00:00,36532,158745,26027,11468,212599,20173,0.214075,4.032782e+09,1.616832e+10,2492


In [93]:
gb_total.to_csv('gb_total.csv')

# 3. Агрегируем по часам/пользователям

In [4]:
dfs = []
for i in tqdm(range(1, 10)):
    dfs.append(
        pd.read_csv(f'parsed_radius_log_{i}.csv')
        [['date', 'логин_абонента', 'type_', 'start_stop_alige', 'status_ok_or_not', 'delay', 'input_trafic', 'output_trafic']]
    )
dfs_concat = pd.concat(dfs)

In [12]:
dfs_concat['day-hour'] = [x[5:13] for x in tqdm(dfs_concat['date'].values)]

In [14]:
dfs_concat['stop'] = (dfs_concat['start_stop_alige'] == 'Stop').astype(int)

In [16]:
dfs_concat['trafic_input_zero'] = (dfs_concat['input_trafic'] == 0).astype(int)
dfs_concat['trafic_output_zero'] = (dfs_concat['output_trafic'] == 0).astype(int)

In [15]:
dfs_concat['failed'] = [int('FAILED' in x) for x in tqdm(dfs_concat['status_ok_or_not'].values)]

In [17]:
dfs_concat.head()

,date,логин_абонента,type_,start_stop_alige,status_ok_or_not,delay,input_trafic,output_trafic,day-hour,stop,failed,trafic_input_zero,trafic_output_zero
0,2020-05-02 00:11:14,rSTKIoB9@pppoe,Billing-Accounting,Alive,OK,0.357546,1.080459e+10,1.719260e+11,05-02 00,0,0,0,0
1,2020-05-02 00:11:14,GBRFnJAM@pppoe,Billing-Accounting,Alive,OK,0.273068,2.989776e+06,7.382877e+07,05-02 00,0,0,0,0
2,2020-05-02 00:11:14,OAXHLWA9@pppoe,Billing-Accounting,Alive,OK,0.240214,5.977762e+09,5.027136e+10,05-02 00,0,0,0,0
3,2020-05-02 00:11:14,GBRFnJAM@pppoe,Billing-Accounting,Alive,OK,0.280033,9.250671e+06,3.035413e+08,05-02 00,0,0,0,0
4,2020-05-02 00:11:14,Wssgg9-W@pppoe,Billing-Accounting,Alive,OK,0.282596,2.394997e+06,7.930324e+07,05-02 00,0,0,0,0


In [42]:
dfs_concat.isnull().sum()

date                         0
логин_абонента               0
type_                        0
start_stop_alige       9783898
status_ok_or_not             0
delay                  4795580
input_trafic          19509616
output_trafic         19509616
day-hour                     0
stop                         0
failed                       0
trafic_input_zero            0
trafic_output_zero           0
dtype: int64

In [22]:
gb_userlogin_hour = (
    dfs_concat
    .groupby(['логин_абонента', 'day-hour'])
    [['stop', 'failed', 'trafic_input_zero', 'trafic_output_zero', 'input_trafic', 'output_trafic']]
    .sum()
)

In [25]:
gb_userlogin_hour.reset_index().to_csv('gb_userlogin_hour.csv', index=False)

In [23]:
gb_userlogin_hour.shape

(10582045, 6)

In [40]:
gb_userlogin_hour.reset_index()['логин_абонента'].str.split('@').str[1].value_counts()

dhcp             5562339
pppoe            5008320
CEbj.Cz.6V         10581
CEbj.dI6Ed.6V        605
wifiul               130
FbmbVj                51
startip_nn            15
FFF.Cz.6V              4
Name: логин_абонента, dtype: int64

In [41]:
gb_userlogin_hour.tail().reset_index()

,логин_абонента,day-hour,stop,failed,trafic_input_zero,trafic_output_zero,input_trafic,output_trafic
0,zzl1d098@pppoe,05-02 14,0,0,0,0,5.193323e+09,5.850485e+10
1,zzl1d098@pppoe,05-02 16,0,0,0,0,5.499853e+09,6.717188e+10
2,zzl1d098@pppoe,05-02 18,0,0,0,0,5.514587e+09,6.744701e+10
3,zzl1d098@pppoe,05-02 20,0,0,0,0,5.341497e+09,6.171147e+10
4,zzl1d098@pppoe,05-02 22,0,0,0,0,5.341602e+09,6.171189e+10


In [ ]:
'логин_абонента', 'day-hour', 'stop', 'failed', 'trafic_input_zero', 'trafic_output_zero', 'input_trafic', 'output_trafic'

In [21]:
dfs_concat['trafic_output_zero'].mean()

0.0008548113218790042

In [10]:
dfs_concat.iloc[0,0][5:13]

'05-02 00'

In [33]:
gb_userlogin_hour['failed'].value_counts()

0      10531857
11         9875
9          7071
4          6302
12         5804
         ...   
174           1
175           1
154           1
140           1
214           1
Name: failed, Length: 232, dtype: int64